In a regression problem, we aim to predict the output of a continuous value, like a price or a probability.
This notebook uses the classic Automobile Dataset and builds a model to predict the fuel efficiency of 1985 year automobiles. To do this, we'll provide the model with a description of many automobiles from that time period. This description includes attributes like: cylinders, engine-size, horsepower, and weight.

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

In [ ]:
dataset_path = keras.utils.get_file("auto.data", "http://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data")
dataset_path

In [ ]:
column_names = ['symboling', 'normalized-losses', 'make', 'fuel-type', 'aspiration', 'num-of-doors', 'body-style', 'drive-wheels', 'engine-location', 'wheel-base', 'length', 'width', 'height', 'curb-weight', 'engine-type', 'num-of-cylinders', 'engine-size', 'fuel-system', 'bore', 'stroke', 'compression-ratio', 'horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg', 'price']
raw_dataset = pd.read_csv(dataset_path, names=column_names,
                      na_values = "?", comment='\t',
                      sep=",", skipinitialspace=True)

dataset = raw_dataset.copy()
dataset.tail()

In [ ]:
dataset.isna().sum()

In [ ]:
dataset = dataset.dropna()

In [ ]:
origin = dataset.pop('fuel-type')
dataset['gas'] = (origin == 'gas')*1.0
dataset['diesel'] = (origin == 'diesel')*1.0

In [ ]:
aspir = dataset.pop('aspiration')
dataset['std'] = (aspir == 'std')*1.0
dataset['turbo'] = (aspir == 'turbo')*1.0

In [ ]:
dw = dataset.pop('drive-wheels')
dataset['4wd'] = (dw == '4wd')*1.0
dataset['fwd'] = (dw == 'fwd')*1.0
dataset['rwd'] = (dw == 'rwd')*1.0

In [ ]:
def score_to_numeric(r):
    if r=='two':
        return 2
    if r=='three':
        return 3
    if r=='four':
        return 4
    if r=='five':
        return 5
    if r=='six':
        return 6
    if r=='eight':
        return 8
    if r=='twelve':
        return 12

In [ ]:
dataset['num-of-doors'] = dataset['num-of-doors'].apply(score_to_numeric)
dataset['num-of-cylinders'] = dataset['num-of-cylinders'].apply(score_to_numeric)

In [ ]:
del dataset['symboling']
del dataset['make']
del dataset['body-style']
del dataset['engine-location']
del dataset['engine-type']
del dataset['fuel-system']

In [ ]:
dataset.head()

In [ ]:
dataset = pd.DataFrame(data=dataset, dtype=np.float64)

In [ ]:
train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)

In [ ]:
from pandas.plotting import scatter_matrix
df = pd.DataFrame(data=train_dataset, columns=['city-mpg', 'num-of-cylinders', 'engine-size', 'curb-weight'])
scatter_matrix(df, figsize=(6, 6), diagonal='kde')

In [ ]:
train_stats = train_dataset.describe()
train_stats.pop("city-mpg")
train_stats = train_stats.transpose()
train_stats

In [ ]:
train_labels = train_dataset.pop('city-mpg')
test_labels = test_dataset.pop('city-mpg')

In [ ]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

In [ ]:
def build_model():
  model = keras.Sequential([
    layers.Dense(128, activation=tf.nn.relu, input_shape=[len(train_dataset.keys())]),
    layers.Dense(128, activation=tf.nn.relu),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mean_squared_error',
                optimizer=optimizer,
                metrics=['mean_absolute_error', 'mean_squared_error'])
  return model

In [ ]:
model = build_model()

In [ ]:
model.summary()

In [ ]:
example_batch = normed_train_data[:10]
example_result = model.predict(example_batch)
example_result

In [ ]:
# Display training progress by printing a single dot for each completed epoch
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print(':)', end='')

EPOCHS = 1000

history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[PrintDot()])

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error [city-mpg]')
  plt.plot(hist['epoch'], hist['mean_absolute_error'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mean_absolute_error'],
           label = 'Val Error')
  plt.ylim([0,30])
  plt.legend()

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Square Error [$city-mpg^2$]')
  plt.plot(hist['epoch'], hist['mean_squared_error'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mean_squared_error'],
           label = 'Val Error')
  plt.ylim([0,100])
  plt.legend()
  plt.show()


plot_history(history)

In [ ]:
model = build_model()

# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(normed_train_data, train_labels, epochs=EPOCHS,
                    validation_split = 0.2, verbose=0, callbacks=[early_stop, PrintDot()])

plot_history(history)

In [ ]:
loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=0)

print("Testing set Mean Abs Error: {:5.2f} city-mpg".format(mae))

In [ ]:
test_predictions = model.predict(normed_test_data).flatten()

plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [city-mpg]')
plt.ylabel('Predictions [city-mpg]')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_ = plt.plot([-100, 100], [-100, 100])

In [ ]:
error = test_predictions - test_labels
plt.hist(error, bins = 25)
plt.xlabel("Prediction Error [city-mpg]")
_ = plt.ylabel("Count")